In [1]:
# libraries
import pandas as  pd
from difflib import SequenceMatcher
import os
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter


# function to match most similar str from list and return percentage of match
def get_string_similarity(a, b):
    # Normalize strings
    a = a.lower().strip()
    b = b.lower().strip()
    # Remove special characters
    a = ''.join(e for e in a if e.isalnum() or e.isspace())
    b = ''.join(e for e in b if e.isalnum() or e.isspace())
    return SequenceMatcher(None, a, b).ratio()

# Clean column names while preserving GUID
def clean_column_name(col):
    if col == 'GUID':
        return col
    return col.split('\n')[0].strip()

# functino to match family to element in reference data
def match_famiglia_to_elemento(df_reference, df_exported):
    """
    Match Famiglia values to ELEMENTO values using string similarity
    
    Parameters:
    df_reference: DataFrame containing reference data with ELEMENTO column
    df_exported: DataFrame containing exported data with Famiglia column
    
    Returns:
    Dictionary of matches {famiglia: best_matching_elemento} or None if no Famiglia column
    """
    # Check if Famiglia column exists
    if 'Famiglia' not in df_exported.columns:
        return None
        
    # Get unique ELEMENTI from reference
    elementi_list = df_reference['ELEMENTO'].unique().tolist()
    
    # Get unique Famiglia value from exported data
    famiglia = df_exported['Famiglia'].iloc[0]  # Assuming same value in all rows
    
    # Find best match
    best_match = max(elementi_list, 
                    key=lambda x: get_string_similarity(famiglia, x))
    
    similarity = get_string_similarity(famiglia, best_match)
    
    return {
        'Famiglia': famiglia,
        'Matched_Elemento': best_match,
        'Similarity_Score': similarity
    }
    
# checks after str cleaning if the parameters required (from df_reference matched with element) are present if not prints what it is missing
def check_parameters(required_params, actual_params):
    
    # Normalize strings in both lists - convert to lowercase and strip whitespace
    required_set = {str(p).lower().strip() for p in required_params}
    actual_set = {str(p).lower().strip() for p in actual_params}

    # Check if all required parameters are present
    missing_params = required_set - actual_set
    all_present = len(missing_params) == 0

    return {
        'all_parameters_present': all_present,
        'missing_parameters': sorted(list(missing_params))
    }
    
# Clean column names while preserving GUID
def clean_column_name(col):
    if col == 'GUID':
        return col
    return col.split('\n')[0].strip()


# imports reference data from excel file and cleans it (removes some columns and rows thought to not be needed)
def import_reference_data(file_path='data/reference.xlsx', sheet_name='Modello_Dati'):
    """
    Import and clean reference data from Excel file.
    
    Parameters:
        file_path (str): Path to reference Excel file
        sheet_name (str): Name of sheet containing reference data
    
    Returns:
        pandas.DataFrame: Cleaned reference data
    """
    try:
        # Read reference Excel file
        df_reference = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=4)
        print(f"Successfully loaded reference data from {file_path}")
        # Drop empty rows from the PE column
        df_reference = df_reference.dropna(subset=['PE'])
        # drop columns not needed drom list
        to_drop = ['Unnamed: 0','PROGETTO','FAMIGLIA','DOCFAP','PFTE','PED','PE', 'ASB']
        #dropping the columns
        df_reference = df_reference.drop(columns=to_drop)
        return df_reference
        
    except Exception as e:
        print(f"Error loading reference data: {str(e)}")
        return None

# imports to be tested data from excel file and cleans it
def test_data_import(file_path='data/export.xlsx'):
    """
    Test function for importing and cleaning Excel data.
    
    Parameters:
        file_path (str): Path to test Excel file
    
    Returns:
        dict: Test results with processed dataframes and validation info
    """
    try:
        # Validate file existence
        if not os.path.exists(file_path):
            return {'success': False, 'error': f'File not found: {file_path}'}

        # Load Excel file
        full_excel = pd.ExcelFile(file_path)
        sheet_names = full_excel.sheet_names
        df_creati = []
        test_results = {
            'success': True,
            'processed_sheets': [],
            'skipped_sheets': [],
            'errors': [],
            'dataframes': []
        }

        for sheet in sheet_names:
            try:
                # Skip specific sheets
                if sheet in ['Instructions', 'ParamValues']:
                    test_results['skipped_sheets'].append(sheet)
                    continue

                # Determine rows to skip
                skip_number = 2 if sheet == 'Zone riscaldamento, ventilazion' else 1

                # Process sheet
                df = full_excel.parse(sheet, skiprows=skip_number)
                
                # Validate dataframe
                if df.empty:
                    test_results['errors'].append(f'Empty dataframe in sheet: {sheet}')
                    continue

                # Clean columns
                df.columns = [clean_column_name(col) for col in df.columns]
                df = df.dropna(axis=1, how='all')

                # Store results
                test_results['processed_sheets'].append(sheet)
                test_results['dataframes'].append(df)

            except Exception as e:
                test_results['errors'].append(f'Error in sheet {sheet}: {str(e)}')

        return test_results

    except Exception as e:
        return {'success': False, 'error': str(e)}

df_reference = import_reference_data()
test_results = test_data_import()
df_creati = test_results['dataframes']
print(df_reference, test_results)

Successfully loaded reference data from data/reference.xlsx
                                      OPERA                    PARTE D'OPERA  \
0                                Cavalcavia  Impalcato Acciaio e Calcesruzzo   
1                                Cavalcavia  Impalcato Acciaio e Calcesruzzo   
2                                Cavalcavia  Impalcato Acciaio e Calcesruzzo   
3                                Cavalcavia  Impalcato Acciaio e Calcesruzzo   
4                                Cavalcavia  Impalcato Acciaio e Calcesruzzo   
...                                     ...                              ...   
17233  Fabbricati Tecnologici e Viaggiatori        Elettrico Apparecchiature   
17234  Fabbricati Tecnologici e Viaggiatori        Elettrico Apparecchiature   
17235  Fabbricati Tecnologici e Viaggiatori        Elettrico Apparecchiature   
17236  Fabbricati Tecnologici e Viaggiatori        Elettrico Apparecchiature   
17237  Fabbricati Tecnologici e Viaggiatori        Elettrico

In [22]:
# vecchia funzione import test data
def test_data_import(file_path):
    """
    Imports and cleans data from an Excel file based on a reference.

    Parameters:
        file_path (str): Path to the Excel file.
    
    Returns:
        list: A list of processed dataframes.
    """
    # Read the Excel file
    full_excel = pd.ExcelFile(file_path)
    sheet_names = full_excel.sheet_names
    df_creati = []

    for sheet in sheet_names:
        print(f"Processing sheet: {sheet}")
        
        if sheet == 'Instructions' or sheet == 'ParamValues':
            print(f"Skipping non-needed sheet: {sheet}")
            continue

        skip_number = 1
        if sheet == 'Zone riscaldamento, ventilazion':
            skip_number = 2

        try:
            df = full_excel.parse(sheet, skiprows=skip_number)
            df.columns = [clean_column_name(col) for col in df.columns]
            df = df.dropna(axis=1, how='all')
            
            sheet_name = sheet.strip().replace(',', '').lower()
            globals()[sheet_name] = df
            df_creati.append(df)
            print(f"Successfully processed sheet: {sheet}")

        except Exception as e:
            print(f"Error processing sheet {sheet}: {str(e)}")

    return df_creati

df_creati = test_data_import('data/export.xlsx')

Processing sheet: Finestre
Successfully processed sheet: Finestre
Processing sheet: Fondazioni strutturali
Successfully processed sheet: Fondazioni strutturali
Processing sheet: Muri
Successfully processed sheet: Muri
Processing sheet: Pavimenti
Successfully processed sheet: Pavimenti
Processing sheet: Pilastri strutturali
Successfully processed sheet: Pilastri strutturali
Processing sheet: Porte
Successfully processed sheet: Porte
Processing sheet: Tetti
Successfully processed sheet: Tetti
Processing sheet: Zone riscaldamento, ventilazion
Successfully processed sheet: Zone riscaldamento, ventilazion
Processing sheet: Instructions
Skipping non-needed sheet: Instructions
Processing sheet: ParamValues
Skipping non-needed sheet: ParamValues


In [2]:
# explorative look at what Famiglia matches what ELEMENTO
for df in df_creati:
    match = match_famiglia_to_elemento(df_reference, df)
    if match:
        print(match)
    else:
        print(f"Skipping dataframe - No 'Famiglia' column found")

{'Famiglia': 'Finestra Doppia', 'Matched_Elemento': 'Anta Doppia Battente', 'Similarity_Score': 0.5714285714285714}
{'Famiglia': 'Plinto', 'Matched_Elemento': 'Plinti', 'Similarity_Score': 0.8333333333333334}
{'Famiglia': 'Muro di base', 'Matched_Elemento': 'Muro d’ala', 'Similarity_Score': 0.6666666666666666}
{'Famiglia': 'Pavimento', 'Matched_Elemento': 'Pavimentazione', 'Similarity_Score': 0.782608695652174}
{'Famiglia': 'Pilastro in calcestruzzo - Rettangolare', 'Matched_Elemento': 'Scambiatore di calore', 'Similarity_Score': 0.4406779661016949}
{'Famiglia': 'Scorrevole Semplice', 'Matched_Elemento': 'Anta Scoorevole', 'Similarity_Score': 0.5294117647058824}
{'Famiglia': 'Tetto di base', 'Matched_Elemento': 'Lettore badge', 'Similarity_Score': 0.6153846153846154}
Skipping dataframe - No 'Famiglia' column found


In [3]:
# version with IFC classes instead of ELEMENTO and Famiglia
def match_ifc_class(df_reference, df_exported):
    """
    Match IFC CLASS values to IFCCLAS values using string similarity
    
    Parameters:
    df_reference: DataFrame containing reference data with ELEMENTO column
    df_exported: DataFrame containing exported data with Famiglia column
    
    Returns:
    Dictionary of matches {famiglia: best_matching_elemento} or None if no Famiglia column
    """
    # Check if Famiglia column exists
    if 'Esporta in formato IFC con nome' not in df_exported.columns:
        return None
        
    # Get unique ELEMENTI from reference
    elementi_list = df_reference['CLASSE IFC'].dropna().unique().tolist()
    
    # Get unique Famiglia value from exported data
    famiglia = df_exported['Esporta in formato IFC con nome'].iloc[0]  # Assuming same value in all rows

    # Find best match
    best_match = max(elementi_list, 
                    key=lambda x: get_string_similarity(famiglia, x))
    
    similarity = get_string_similarity(famiglia, best_match)
    
    return {
        'Classe IFC': famiglia,
        'Matched_Elemento': best_match,
        'Similarity_Score': similarity
    }

# explorative look at what family matches what element
for df in df_creati:
    match = match_ifc_class(df_reference, df)
    if match:
        print(dict(match))
    else:
        print(f"Skipping dataframe - No 'IFC' column found")
    

{'Classe IFC': 'IfcWindow', 'Matched_Elemento': 'IfcWindow', 'Similarity_Score': 1.0}
{'Classe IFC': 'IfcFooting', 'Matched_Elemento': 'IfcFooting', 'Similarity_Score': 1.0}
{'Classe IFC': 'IfcWall', 'Matched_Elemento': 'IfcWall', 'Similarity_Score': 1.0}
{'Classe IFC': 'IfcSlab', 'Matched_Elemento': 'IfcSlab', 'Similarity_Score': 1.0}
{'Classe IFC': 'IfcColumn', 'Matched_Elemento': 'IfcColumn', 'Similarity_Score': 1.0}
{'Classe IFC': 'IfcDoor', 'Matched_Elemento': 'IfcDoor', 'Similarity_Score': 1.0}
{'Classe IFC': 'IfcSlab', 'Matched_Elemento': 'IfcSlab', 'Similarity_Score': 1.0}
Skipping dataframe - No 'IFC' column found


In [ ]:
# function to check if the parameters required (from df_reference matched with element) are present if not prints what it is missing

def check_parameters(required_params, actual_params):
    # Normalize strings in both lists - convert to lowercase and strip whitespace
    required_set = {str(p).lower().strip() for p in required_params}
    actual_set = {str(p).lower().strip() for p in actual_params}
    
    # Check if all required parameters are present
    missing_params = required_set - actual_set
    all_present = len(missing_params) == 0
    
    return {
        'all_parameters_present': all_present,
        'missing_parameters': sorted(list(missing_params))
    }

def check_df_parameters(df_reference, df_export):
    """Check parameters for a single dataframe against reference"""
    # Get famiglia match
    match = match_famiglia_to_elemento(df_reference, df_export)
    if not match:
        return f"Skipping dataframe - No 'Famiglia' column found"
    
    # Get parameters to check
    famiglia = match['Famiglia']
    elemento = match['Matched_Elemento']
    #df_elemento = df_reference[df_reference['ELEMENTO'] == elemento]
    param_check = list(df_reference[df_reference['ELEMENTO'] == elemento]['PARAMETRI INFORMATIVI'])
    param_current = list(df_export.columns)
    
    # Check parameters
    result = check_parameters(param_check, param_current)
    
    # Format output
    output = f"\nChecking parameters for Famiglia: {famiglia} (matched to {elemento})"
    output += f"\nAll parameters present: {result['all_parameters_present']}"
    if not result['all_parameters_present']:
        output += "\nMissing parameters:"
        for param in result['missing_parameters']:
            output += f"\n- {param}"
    return output

# Check all dataframes
for df in df_creati:
    result = check_df_parameters(df_reference, df)
    print(result)


Checking parameters for Famiglia: Finestra Doppia (matched to Anta Doppia Battente)
All parameters present: False
Missing parameters:
- codifica categoria
- descrizione disciplina

Checking parameters for Famiglia: Plinto (matched to Plinti)
All parameters present: False
Missing parameters:
- caratteristiche terreno
- codifica categoria
- descrizione disciplina

Checking parameters for Famiglia: Muro di base (matched to Muro d’ala)
All parameters present: False
Missing parameters:
- codifica categoria
- descrizione disciplina

Checking parameters for Famiglia: Pavimento (matched to Pavimentazione)
All parameters present: False
Missing parameters:
- caratterisiche generiche
- codifica categoria
- descrizione disciplina
- tipologia strato

Checking parameters for Famiglia: Pilastro in calcestruzzo - Rettangolare (matched to Scambiatore di calore)
All parameters present: False
Missing parameters:
- capacità
- codifica categoria
- descrizione disciplina
- diametro di copertura - antincend

In [ ]:
# versione funzionante con colori, crea un file excel per ogni famiglia di elementi analizzata 
def check_df_parameters(df_reference, df_export):
    """Check parameters for a single dataframe against reference and export a full comparison Excel file with all df_elemento columns."""
    # Create results directory if it doesn't exist
    os.makedirs('results', exist_ok=True)
    
    match = match_famiglia_to_elemento(df_reference, df_export)
    if not match:
        return f"Skipping dataframe - No 'Famiglia' column found"
    
    famiglia = match['Famiglia']
    elemento = match['Matched_Elemento']
    
    # Get the rows in df_reference corresponding to the matched ELEMENTO
    df_elemento = df_reference[df_reference['ELEMENTO'] == elemento]
    param_check = list(df_elemento['PARAMETRI INFORMATIVI'])
    param_current = list(df_export.columns)
    
    # Create a copy and add our additional columns
    df_comparison = df_elemento.copy()
    df_comparison['parameter_exists'] = df_comparison['PARAMETRI INFORMATIVI'].isin(param_current)
    
    # Create mapping dictionary using normalized matching
    param_mapping = {}
    for ref_param in df_comparison['PARAMETRI INFORMATIVI']:
        for curr_param in param_current:
            if ref_param.lower().strip() == curr_param.lower().strip():
                param_mapping[ref_param] = curr_param
                break
            
    df_comparison['param_current'] = df_comparison['PARAMETRI INFORMATIVI'].map(param_mapping)
    
    # Define fill colors
    green_fill = PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid')
    red_fill = PatternFill(start_color='FFB6C1', end_color='FFB6C1', fill_type='solid')
    
    # Save full df_comparison (with all original columns plus our added ones)
    filename = os.path.join('results', f'parameter_check_{famiglia.replace(" ", "_")}.xlsx')
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        
        #df_comparison= df_comparison.drop(columns=['PE'])
        df_comparison.to_excel(writer, sheet_name='Parameter Check', index=False)
        
        worksheet = writer.sheets['Parameter Check']
        
        # Auto-adjust every column's width
        for idx, col in enumerate(df_comparison.columns, start=1):
            max_length = max(df_comparison[col].astype(str).apply(len).max(), len(col))
            worksheet.column_dimensions[get_column_letter(idx)].width = max_length + 2
        
        # Find the column letter for "parameter_exists"
        param_exists_index = list(df_comparison.columns).index('parameter_exists') + 1
        col_letter = get_column_letter(param_exists_index)
        
        # Apply fill colors for the "parameter_exists" column for each row
        for row in range(2, len(df_comparison) + 2):  # data starts from row 2
            cell = worksheet[f'{col_letter}{row}']
            # Access current value using df_comparison's index (row-2)
            if df_comparison.iloc[row - 2]['parameter_exists']:
                cell.fill = green_fill
            else:
                cell.fill = red_fill
    
    return filename

# Check all dataframes and save Excel reports showing full comparison (including all df_elemento columns)
for df in df_creati:
    try:
        output_file = check_df_parameters(df_reference, df)
        if isinstance(output_file, str) and output_file.startswith('Skipping'):
            print(output_file)
        else:
            print(f"Results saved to: {output_file}")
    except Exception as e:
        print(f"Error processing dataframe: {str(e)}")
    print("-" * 50)

Results saved to: results\parameter_check_Finestra_Doppia.xlsx
--------------------------------------------------
Results saved to: results\parameter_check_Plinto.xlsx
--------------------------------------------------
Results saved to: results\parameter_check_Muro_di_base.xlsx
--------------------------------------------------
Results saved to: results\parameter_check_Pavimento.xlsx
--------------------------------------------------
Results saved to: results\parameter_check_Pilastro_in_calcestruzzo_-_Rettangolare.xlsx
--------------------------------------------------
Results saved to: results\parameter_check_Scorrevole_Semplice.xlsx
--------------------------------------------------
Results saved to: results\parameter_check_Tetto_di_base.xlsx
--------------------------------------------------
Skipping dataframe - No 'Famiglia' column found
--------------------------------------------------


In [ ]:
# versione funzionante con colori, crea una nuova sheet per ogni famiglia di elementi analizzata e crea summary finale
# libraries
import pandas as  pd
from difflib import SequenceMatcher
import os
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter

# function to match most similar str from list and return percentage of match
def get_string_similarity(a, b):
    # Normalize strings
    a = a.lower().strip()
    b = b.lower().strip()
    # Remove special characters
    a = ''.join(e for e in a if e.isalnum() or e.isspace())
    b = ''.join(e for e in b if e.isalnum() or e.isspace())
    return SequenceMatcher(None, a, b).ratio()

# Clean column names while preserving GUID
def clean_column_name(col):
    if col == 'GUID':
        return col
    return col.split('\n')[0].strip()

# functino to match family to element in reference data
def match_famiglia_to_elemento(df_reference, df_exported):
    """
    Match Famiglia values to ELEMENTO values using string similarity
    
    Parameters:
    df_reference: DataFrame containing reference data with ELEMENTO column
    df_exported: DataFrame containing exported data with Famiglia column
    
    Returns:
    Dictionary of matches {famiglia: best_matching_elemento} or None if no Famiglia column
    """
    # Check if Famiglia column exists
    if 'Famiglia' not in df_exported.columns:
        return None
        
    # Get unique ELEMENTI from reference
    elementi_list = df_reference['ELEMENTO'].unique().tolist()
    
    # Get unique Famiglia value from exported data
    famiglia = df_exported['Famiglia'].iloc[0]  # Assuming same value in all rows
    
    # Find best match
    best_match = max(elementi_list, 
                    key=lambda x: get_string_similarity(famiglia, x))
    
    similarity = get_string_similarity(famiglia, best_match)
    
    return {
        'Famiglia': famiglia,
        'Matched_Elemento': best_match,
        'Similarity_Score': similarity
    }
    
# checks after str cleaning if the parameters required (from df_reference matched with element) are present if not prints what it is missing
def check_parameters(required_params, actual_params):
    
    # Normalize strings in both lists - convert to lowercase and strip whitespace
    required_set = {str(p).lower().strip() for p in required_params}
    actual_set = {str(p).lower().strip() for p in actual_params}

    # Check if all required parameters are present
    missing_params = required_set - actual_set
    all_present = len(missing_params) == 0

    return {
        'all_parameters_present': all_present,
        'missing_parameters': sorted(list(missing_params))
    }
    
# Clean column names while preserving GUID
def clean_column_name(col):
    if col == 'GUID':
        return col
    return col.split('\n')[0].strip()

# imports reference data from excel file and cleans it (removes some columns and rows thought to not be needed)
def import_reference_data(file_path, sheet_name='Modello_Dati'):
    """
    Import and clean reference data from Excel file.
    
    Parameters:
        file_path (str): Path to reference Excel file
        sheet_name (str): Name of sheet containing reference data
    
    Returns:
        pandas.DataFrame: Cleaned reference data
    """
    try:
        # Read reference Excel file
        df_reference = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=4)
        print(f"Successfully loaded reference data from {file_path}")
        # Drop empty rows from the PE column
        df_reference = df_reference.dropna(subset=['PE'])
        # drop columns not needed drom list
        to_drop = ['Unnamed: 0','PROGETTO','FAMIGLIA','DOCFAP','PFTE','PED','PE', 'ASB']
        #dropping the columns
        df_reference = df_reference.drop(columns=to_drop)
        return df_reference
        
    except Exception as e:
        print(f"Error loading reference data: {str(e)}")
        return None

# imports to be tested data from excel file and cleans it
def import_test_data(file_path):
    """
    Test function for importing and cleaning Excel data.
    
    Parameters:
        file_path (str): Path to test Excel file
    
    Returns:
        dict: Test results with processed dataframes and validation info
    """
    try:
        # Validate file existence
        if not os.path.exists(file_path):
            return {'success': False, 'error': f'File not found: {file_path}'}

        # Load Excel file
        full_excel = pd.ExcelFile(file_path)
        sheet_names = full_excel.sheet_names
        df_creati = []
        test_results = {
            'success': True,
            'processed_sheets': [],
            'skipped_sheets': [],
            'errors': [],
            'dataframes': []
        }

        for sheet in sheet_names:
            try:
                # Skip specific sheets
                if sheet in ['Instructions', 'ParamValues']:
                    test_results['skipped_sheets'].append(sheet)
                    continue

                # Determine rows to skip
                skip_number = 2 if sheet == 'Zone riscaldamento, ventilazion' else 1

                # Process sheet
                df = full_excel.parse(sheet, skiprows=skip_number)
                
                # Validate dataframe
                if df.empty:
                    test_results['errors'].append(f'Empty dataframe in sheet: {sheet}')
                    continue

                # Clean columns
                df.columns = [clean_column_name(col) for col in df.columns]
                df = df.dropna(axis=1, how='all')

                # Store results
                test_results['processed_sheets'].append(sheet)
                test_results['dataframes'].append(df)

            except Exception as e:
                test_results['errors'].append(f'Error in sheet {sheet}: {str(e)}')

        return test_results

    except Exception as e:
        return {'success': False, 'error': str(e)}

# function that writes the checked parameters to an excel file
def check_df_parameters(df_reference, df_export, writer, missing_params_summary):
    """Check parameters for a single dataframe and write to shared Excel writer."""
    match = match_famiglia_to_elemento(df_reference, df_export) # function to match famiglia to elemento returns a dict or None
    if not match:
        return "Skipping dataframe - No 'Famiglia' column found"
    
    famiglia = match['Famiglia']
    elemento = match['Matched_Elemento']
    
    df_elemento = df_reference[df_reference['ELEMENTO'] == elemento]
    param_current = list(df_export.columns)
    
    df_comparison = df_elemento.copy()
    df_comparison['parameter_exists'] = df_comparison['PARAMETRI INFORMATIVI'].isin(param_current)
    
    param_mapping = {}
    for ref_param in df_comparison['PARAMETRI INFORMATIVI']:
        for curr_param in param_current:
            if ref_param.lower().strip() == curr_param.lower().strip():
                param_mapping[ref_param] = curr_param
                break
    
    df_comparison['param_current'] = df_comparison['PARAMETRI INFORMATIVI'].map(param_mapping)
    
    # Track missing parameters for summary
    missing_params = df_comparison[~df_comparison['parameter_exists']]
    if not missing_params.empty:
        missing_params_summary.append({
            'Famiglia': famiglia,
            'Elemento': elemento,
            'Missing Parameters': ', '.join(missing_params['PARAMETRI INFORMATIVI'].tolist())
        })
    
    # Write to sheet named after famiglia
    sheet_name = f"{famiglia[:31]}"  # Excel sheet names limited to 31 chars
    df_comparison.to_excel(writer, sheet_name=sheet_name, index=False)
    
    # Format sheet
    worksheet = writer.sheets[sheet_name]
    # Auto-adjust columns
    for idx, col in enumerate(df_comparison.columns, start=1):
        max_length = max(df_comparison[col].astype(str).apply(len).max(), len(col))
        worksheet.column_dimensions[get_column_letter(idx)].width = max_length + 2
    
    # Apply colors to parameter_exists column
    param_exists_index = list(df_comparison.columns).index('parameter_exists') + 1
    col_letter = get_column_letter(param_exists_index)
    
    for row in range(2, len(df_comparison) + 2):
        cell = worksheet[f'{col_letter}{row}']
        if df_comparison.iloc[row - 2]['parameter_exists']:
            cell.fill = green_fill
        else:
            cell.fill = red_fill
            
    return f"Processed {famiglia}"

# function to create a summary sheet with missing parameters
def create_summary_sheet(writer, missing_params_summary):
    """
    Creates and formats a summary sheet in the Excel workbook showing missing parameters.
    
    Args:
        writer: ExcelWriter object
        missing_params_summary: List of dicts containing missing parameter info
    """
    if not missing_params_summary:
        return
        
    # Create summary DataFrame
    summary_df = pd.DataFrame(missing_params_summary)
    summary_df.to_excel(writer, sheet_name='Missing Parameters Summary', index=False)
    
    # Get and format worksheet
    worksheet = writer.sheets['Missing Parameters Summary']
    
    # Auto-adjust columns
    for idx, col in enumerate(summary_df.columns, start=1):
        max_length = max(summary_df[col].astype(str).apply(len).max(), len(col))
        worksheet.column_dimensions[get_column_letter(idx)].width = max_length + 2
    
    # Color entire rows based on Missing Parameters
    for row in range(2, len(summary_df) + 2):  # Start from 2 to skip header
        missing_params = summary_df.iloc[row-2]['Missing Parameters']
        fill_color = green_fill if pd.isna(missing_params) or missing_params == '' else red_fill
        
        # Apply color to each cell in the row
        for col in range(1, len(summary_df.columns) + 1):
            cell = worksheet[f'{get_column_letter(col)}{row}']
            cell.fill = fill_color

# Import dei dati
refernece_file_path = 'data/reference.xlsx'
export_file_path = 'data/export.xlsx'
# Process all dataframes and save to single Excel file
df_reference = import_reference_data(refernece_file_path)
test_results = import_test_data(export_file_path)
df_creati = test_results['dataframes']

os.makedirs('results', exist_ok=True)
filename = os.path.join('results', 'infrarail_assets_checks.xlsx')
#initializing empty list to store missing parameters it has to be here since it is used populated by check_df_parameters and used by create_summary_sheet
missing_params_summary = []
# Colors for formatting for filling True/False values
green_fill = PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid')
red_fill = PatternFill(start_color='FFB6C1', end_color='FFB6C1', fill_type='solid')

# Usage in main code:
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    for df in df_creati:
        try:
            result = check_df_parameters(df_reference, df, writer, missing_params_summary)
            print(result)
        except Exception as e:
            print(f"Error processing dataframe: {str(e)}")
    
    create_summary_sheet(writer, missing_params_summary)

print(f"\nResults saved to: {filename}")

Successfully loaded reference data from data/reference.xlsx
Processed Finestra Doppia
Processed Plinto
Processed Muro di base
Processed Pavimento
Processed Pilastro in calcestruzzo - Rettangolare
Processed Scorrevole Semplice
Processed Tetto di base
Skipping dataframe - No 'Famiglia' column found

Results saved to: results\infrarail_assets_checks.xlsx
